In [ ]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import read_sas

# 1. Load local NHANES XPT files
# Change paths if your files are in a different folder
tchol_path = "TCHOL_L.xpt"   
demo_path = "DEMO_X.XPT"     

tchol = read_sas(tchol_path, format="xport")
demo = read_sas(demo_path, format="xport")

# 2. Keep needed variables and merge
# TCHOL_L: SEQN (ID), LBXTC (total cholesterol, mg/dL) 
# DEMO_X : SEQN, RIAGENDR (1=male), RIDAGEYR (age), WTMEC2YR (exam weight) 

tchol_small = tchol[["SEQN", "LBXTC"]]
demo_small = demo[["SEQN", "RIAGENDR", "RIDAGEYR", "WTMEC2YR"]]

df = pd.merge(tchol_small, demo_small, on="SEQN", how="inner")

# 3. Filter to males age 40–60 with valid total cholesterol
mask = (
    (df["RIAGENDR"] == 1) &           
    (df["RIDAGEYR"] >= 40) &
    (df["RIDAGEYR"] <= 60) &
    (~df["LBXTC"].isna())
)

subset = df.loc[mask].copy()


# 4. Restrict cholesterol range and create 5 mg/dL bins
min_tc, max_tc, step = 50, 400, 5

subset = subset[(subset["LBXTC"] >= min_tc) & (subset["LBXTC"] <= max_tc)].copy()

bins = np.arange(min_tc, max_tc + step, step)   # 50, 55, ..., 400
labels = bins[:-1] + step / 2.0                 # bin centers: 52.5, 57.5, ...

subset["bin"] = pd.cut(
    subset["LBXTC"],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

subset = subset.dropna(subset=["bin"])

# 5. Compute weighted percentage distribution
weighted_totals = subset.groupby("bin")["WTMEC2YR"].sum()
total_weight = weighted_totals.sum()

population_perc = (weighted_totals / total_weight) * 100.0

dist = population_perc.reset_index()
dist.columns = ["cholesterol_level", "population_perc"]
dist["cholesterol_level"] = dist["cholesterol_level"].astype(float)

# 6. Save CSV (required format)
csv_filename = "cholesterol_distribution.csv"
dist.to_csv(csv_filename, index=False)

print(f"Saved distribution CSV to: {csv_filename}")
print(dist.head())

# 7. Plot distribution with arrow at 184 mg/dL
plt.figure(figsize=(9, 5))
plt.plot(
    dist["cholesterol_level"],
    dist["population_perc"],
    marker="o",
    linestyle="-",
    linewidth=1.5
)

plt.xlabel("Total cholesterol (mg/dL)")
plt.ylabel("Percentage of individuals (%)")
plt.title("NHANES 2021–2022: Males 40–60\nTotal Cholesterol Distribution")

plt.xlim(min_tc, max_tc)

# Arrow at 184 mg/dL
target_tc = 184
nearest_idx = (dist["cholesterol_level"] - target_tc).abs().idxmin()
nearest_x = dist.loc[nearest_idx, "cholesterol_level"]
nearest_y = dist.loc[nearest_idx, "population_perc"]

y_offset = dist["population_perc"].max() * 0.15

plt.annotate(
    f"{target_tc} mg/dL",
    xy=(nearest_x, nearest_y),
    xytext=(nearest_x + 20, nearest_y + y_offset),
    arrowprops=dict(arrowstyle="->", color="red", linewidth=2),
    color="red",
    fontsize=10
)

plt.tight_layout()
plt.show()
